In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('../../datasets/USvideos.csv')

In [2]:
#drop likes, dislikes, comment_count, thumbnail_link
useless_cols = ['likes', 'dislikes', 'comment_count', 'thumbnail_link', 'video_error_or_removed','video_id']
modified_data = data.drop(columns = useless_cols)
print(modified_data.shape)

(40949, 10)


In [3]:
#drop description missing rows
modified_data['description'].replace('', np.nan, inplace=True)
modified_data.dropna(subset=['description'], inplace=True)
print(modified_data.shape)

#concatanate text columns
modified_data['channel_text'] = modified_data.title + ' ' + modified_data.channel_title + ' ' +modified_data.description +'  '+ modified_data.tags
modified_data = modified_data.drop(columns = ['title', 'channel_title', 'description','tags'])
# modified_data

(40379, 10)


In [4]:
#convert times to timestamps
#change the trending_date format to yyyy-mm-dd from yy.dd.mm
modified_data['formatted_trending_date'] = pd.to_datetime(modified_data['trending_date'],format="%y.%d.%m")
modified_data['formatted_publish_time'] = pd.to_datetime(modified_data['publish_time'],format="%Y-%m-%dT%H:%M:%S.%fZ")
modified_data ['difference_d'] = (modified_data['formatted_trending_date'] - modified_data['formatted_publish_time']).astype('timedelta64[D]')
modified_data ['difference_h'] = (modified_data['formatted_trending_date'] - modified_data['formatted_publish_time']).astype('timedelta64[h]')

#US_video=US_video.assign(publish_day=US_video.publish_time.dt.day,publish_month=US_video.publish_time.dt.month,publish_year=US_video.publish_time.dt.year)

modified_data ['published_year'] = modified_data['formatted_publish_time'].dt.year
modified_data ['published_month'] = modified_data['formatted_publish_time'].dt.month
modified_data ['trending_year'] = modified_data['formatted_trending_date'].dt.year
modified_data ['trending_month'] = modified_data['formatted_trending_date'].dt.month

#modified_data ['published_year_month'] = modified_data ['published_year'].astype(str)[2:] + ' ' + modified_data ['published_month'].astype(str) 
#modified_data ['trending_year_month'] = modified_data ['trending_year'].astype(str)[2:4] + ' ' + modified_data ['trending_month'].astype(str)

#drop outliers - past published videos
modified_data = modified_data[modified_data.published_year.isin([2016,2017,2018])]
print(modified_data.shape)

#get small range of view counts
modified_data = modified_data[(modified_data['views'] >= 10000) & (modified_data['views'] <= 50000000)]
print(modified_data.shape)

modified_data['comments_disabled'] = modified_data['comments_disabled'].apply(lambda x: "\"" + str(x) + "\"")
modified_data['ratings_disabled'] = modified_data['ratings_disabled'].apply(lambda x: "\"" + str(x) + "\"")

#ax =sns.countplot(x=modified_data["views"],data=modified_data)
#sns.distplot(modified_data["published_year"], kde=False, color='blue', bins=5)
#ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
#plt.tight_layout()
#plt.figure(figsize=(16,6))
#plt.show()


(40203, 15)
(39308, 15)


In [5]:

#drop un-wanted date columns
modified_data = modified_data.drop(columns = ['trending_date', 'publish_time', 'formatted_trending_date', 'formatted_publish_time'])
modified_data = modified_data.sample(1200)

modified_data.to_csv('../../datasets/USvideos_Processed_train_1200_with-t_desc.csv',index=False, encoding='utf-8')